<a href="https://colab.research.google.com/github/Nickeniklas/stockdata_forecasting/blob/main/stockdata_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock data Forecasting

Using SARIMAX

Fetching with yfinance



In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# set fetch data parameters
ticker = "AAPL"
start_date = "2010-01-01"
end_date = "2020-01-01"


In [4]:
# fetch data
data = yf.download(ticker, start=start_date, end=end_date)
data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.431894,6.446621,6.382906,6.414463,493729600
2010-01-05,6.443017,6.479382,6.409055,6.449629,601904800
2010-01-06,6.340533,6.468564,6.333921,6.443018,552160000
2010-01-07,6.328810,6.371487,6.282827,6.363974,477131200
2010-01-08,6.370885,6.371487,6.283128,6.320394,447610800
...,...,...,...,...,...
2019-12-24,68.732895,68.882809,68.406488,68.834449,48478800
2019-12-26,70.096596,70.113523,68.836885,68.865899,93121200
